# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('/storage/Demo_Gesture/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/storage/Demo_Gesture/Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/storage/Demo_Gesture/Project_data/train'
val_path = '/storage/Demo_Gesture/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [7]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [8]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 6.9996 - categorical_accuracy: 0.3000

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 94s 1s/step - loss: 6.9210 - categorical_accuracy: 0.3005 - val_loss: 8.7402 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-6.97551-0.30015-8.74023-0.30000.h5
Epoch 2/10
67/67 [==============================] - 28s 416ms/step - loss: 4.8372 - categorical_accuracy: 0.3134 - val_loss: 8.8919 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-4.83719-0.31343-8.89195-0.24000.h5
Epoch 3/10
67/67 [==============================] - 30s 442ms/step - loss: 2.8366 - categorical_accuracy: 0.2836 - val_loss: 2.1485 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-2.83657-0.28358-2.14845-0.30000.h5
Epoch 4/10
67/67 [==============================] - 29s 438ms/step - loss: 1.8071 - categorical_accuracy: 0.2836 - val_loss: 1.3704 - val_categorical_accuracy: 0.4400

Epo

In [9]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [13]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_6 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [14]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [15]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.7241 - categorical_accuracy: 0.2031Batch:  67 Index: 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 96s 1s/step - loss: 1.7210 - categorical_accuracy: 0.2000 - val_loss: 1.6107 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-1.72199-0.20211-1.61072-0.21000.h5
Epoch 2/10
67/67 [==============================] - 28s 415ms/step - loss: 1.6102 - categorical_accuracy: 0.1692 - val_loss: 1.6114 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-1.61024-0.16915-1.61143-0.16000.h5
Epoch 3/10
67/67 [==============================] - 31s 456ms/step - loss: 1.6124 - categorical_accuracy: 0.1741 - val_loss: 1.6107 - val_categorical_accuracy: 0.1600

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-1.61241-0.17413-1.61069-0.16000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
67/67 [==============================] - 29s 437ms/step - loss: 1.6090 - catego

### Changing x,y,z values Experiment [ 1]


In [16]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [17]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [18]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [19]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10Source path =  /storage/Demo_Gesture/Project_data/val 
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10
; batch size = 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 1.6703 - categorical_accuracy: 0.1924

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 82s 1s/step - loss: 1.6694 - categorical_accuracy: 0.1896 - val_loss: 1.6092 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-1.66998-0.19155-1.60916-0.21000.h5
Epoch 2/10
67/67 [==============================] - 26s 392ms/step - loss: 1.6110 - categorical_accuracy: 0.1841 - val_loss: 1.6096 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-1.61096-0.18408-1.60955-0.21000.h5
Epoch 3/10
67/67 [==============================] - 25s 372ms/step - loss: 1.6083 - categorical_accuracy: 0.2139 - val_loss: 1.6099 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-1.60831-0.21393-1.60988-0.21000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
67/67 [==============================] - 28s 413ms/step - loss: 1.6081 - catego

### Changing Batch size  to 20 Experiment [ 2]


In [20]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [21]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 20
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 20


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/67 [=============>................] - ETA: 1:39 - loss: 1.3914 - categorical_accuracy: 0.3750Batch:  34 Index: 20


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 105s 2s/step - loss: 1.4096 - categorical_accuracy: 0.3624 - val_loss: 1.3098 - val_categorical_accuracy: 0.4900

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-1.39264-0.37664-1.30983-0.49000.h5
Epoch 2/10
67/67 [==============================] - 29s 431ms/step - loss: 1.4170 - categorical_accuracy: 0.4030 - val_loss: 1.3088 - val_categorical_accuracy: 0.4400

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-1.41703-0.40299-1.30883-0.44000.h5
Epoch 3/10
67/67 [==============================] - 27s 410ms/step - loss: 1.2228 - categorical_accuracy: 0.4726 - val_loss: 1.1473 - val_categorical_accuracy: 0.5600

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-1.22279-0.47264-1.14727-0.56000.h5
Epoch 4/10
67/67 [==============================] - 29s 433ms/step - loss: 1.2401 - categorical_accuracy: 0.4726 - val_loss: 1.1427 - val_categorical_accuracy: 0.6200

Ep

### Changing Batch size  to 30 Experiment [ 3]


In [22]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [23]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 30
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 30


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/67 [>.............................] - ETA: 4:34 - loss: 0.6558 - categorical_accuracy: 0.7333Batch:  4 Index: 30


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 99s 1s/step - loss: 0.8208 - categorical_accuracy: 0.6639 - val_loss: 0.8600 - val_categorical_accuracy: 0.6750

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-0.77600-0.68176-0.86000-0.67500.h5
Epoch 2/10
67/67 [==============================] - 26s 384ms/step - loss: 0.7662 - categorical_accuracy: 0.6716 - val_loss: 0.8829 - val_categorical_accuracy: 0.6600

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-0.76625-0.67164-0.88292-0.66000.h5
Epoch 3/10
67/67 [==============================] - 27s 410ms/step - loss: 0.6805 - categorical_accuracy: 0.6915 - val_loss: 0.9753 - val_categorical_accuracy: 0.7000

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-0.68052-0.69154-0.97528-0.70000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 4/10
67/67 [==============================] - 25s 374ms/step - loss: 0.6333 - catego

### Changing Batch size  to 40 Experiment [ 4]


In [24]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [25]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 40
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 40


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/67 [..............................] - ETA: 6:05 - loss: 0.4529 - categorical_accuracy: 0.8000Batch:  3 Index: 40


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 210s 3s/step - loss: 0.4252 - categorical_accuracy: 0.8347 - val_loss: 0.8867 - val_categorical_accuracy: 0.7083

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-0.42551-0.83478-0.88669-0.70833.h5
Epoch 2/10
67/67 [==============================] - 142s 2s/step - loss: 0.4225 - categorical_accuracy: 0.8405 - val_loss: 0.9108 - val_categorical_accuracy: 0.7050

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-0.42200-0.84034-0.91077-0.70500.h5
Epoch 3/10
67/67 [==============================] - 144s 2s/step - loss: 0.4081 - categorical_accuracy: 0.8385 - val_loss: 0.8104 - val_categorical_accuracy: 0.7300

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-0.40813-0.83845-0.81042-0.73000.h5
Epoch 4/10
67/67 [==============================] - 139s 2s/step - loss: 0.3951 - categorical_accuracy: 0.8516 - val_loss: 0.9578 - val_categorical_accuracy: 0.7000

Epoch 00

### Change Optimizer to Adadelta Experiment [5]

In [26]:
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [27]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [28]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 40
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 40
 2/67 [..............................] - ETA: 6:39 - loss: 0.4547 - categorical_accuracy: 0.8125Batch:  3 Index: 40


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 207s 3s/step - loss: 0.5317 - categorical_accuracy: 0.7966 - val_loss: 0.8815 - val_categorical_accuracy: 0.6625

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-0.54416-0.79304-0.88149-0.66250.h5
Epoch 2/10
67/67 [==============================] - 138s 2s/step - loss: 0.3386 - categorical_accuracy: 0.8775 - val_loss: 0.8501 - val_categorical_accuracy: 0.7100

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-0.34065-0.87639-0.85008-0.71000.h5
Epoch 3/10
67/67 [==============================] - 136s 2s/step - loss: 0.2589 - categorical_accuracy: 0.8999 - val_loss: 1.2603 - val_categorical_accuracy: 0.6350

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-0.25886-0.89991-1.26033-0.63500.h5
Epoch 4/10
67/67 [==============================] - 137s 2s/step - loss: 0.2011 - categorical_accuracy: 0.9298 - val_loss: 1.0706 - val_categorical_accuracy: 0.7150

Epoch 00

### Change epoch to 20 Experiment [6]

In [29]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [30]:
import keras
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


In [ ]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [ ]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Epoch 1/20
Source path =  /storage/Demo_Gesture/Project_data/train Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 40
; batch size = 40
 2/67 [..............................] - ETA: 6:16 - loss: 0.0179 - categorical_accuracy: 1.0000Batch:  3 Index: 40


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 206s 3s/step - loss: 0.0958 - categorical_accuracy: 0.9708 - val_loss: 1.4780 - val_categorical_accuracy: 0.6833

Epoch 00001: saving model to model_init_2018-12-2617_35_26.106810/model-00001-0.10298-0.96812-1.47802-0.68333.h5
Epoch 2/20
67/67 [==============================] - 140s 2s/step - loss: 0.0816 - categorical_accuracy: 0.9716 - val_loss: 1.6118 - val_categorical_accuracy: 0.6750

Epoch 00002: saving model to model_init_2018-12-2617_35_26.106810/model-00002-0.08097-0.97167-1.61185-0.67500.h5
Epoch 3/20
67/67 [==============================] - 140s 2s/step - loss: 0.0485 - categorical_accuracy: 0.9772 - val_loss: 1.7824 - val_categorical_accuracy: 0.6950

Epoch 00003: saving model to model_init_2018-12-2617_35_26.106810/model-00003-0.04847-0.97717-1.78243-0.69500.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/20
67/67 [==============================] - 135s 2s/step - loss: 0.0411 - categorical_accuracy: 0.9877 

#### Till this point we achieved 99.39% accuracy in training data

### Model C Experiment [7]

In [10]:
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(classes, activation='softmax'))
model_c.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_6 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [11]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120# image height
num_epochs = 20

In [12]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [13]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  /storage/Demo_Gesture/Project_data/train Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10
; batch size = 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 3s - loss: 1.8396 - categorical_accuracy: 0.3154Batch:  67 Index: 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 135s 2s/step - loss: 1.8753 - categorical_accuracy: 0.3105 - val_loss: 3.1390 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2018-12-2708_18_49.548506/model-00001-1.85423-0.31373-3.13896-0.29000.h5
Epoch 2/20
67/67 [==============================] - 29s 427ms/step - loss: 1.9485 - categorical_accuracy: 0.2488 - val_loss: 6.3683 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2018-12-2708_18_49.548506/model-00002-1.94852-0.24876-6.36835-0.16000.h5
Epoch 3/20
67/67 [==============================] - 31s 470ms/step - loss: 1.9667 - categorical_accuracy: 0.2438 - val_loss: 2.8393 - val_categorical_accuracy: 0.2900

Epoch 00003: saving model to model_init_2018-12-2708_18_49.548506/model-00003-1.96668-0.24378-2.83926-0.29000.h5
Epoch 4/20
67/67 [==============================] - 28s 425ms/step - loss: 1.7844 - categorical_accuracy: 0.3333 - val_loss: 3.2987 - val_categorical_accuracy: 0.2900

Ep

### Model D Experiment [8]

In [14]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_11 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_11 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_12 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [15]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10
66/67 [============================>.] - ETA: 1s - loss: 5.0497 - categorical_accuracy: 0.2773

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 92s 1s/step - loss: 5.0001 - categorical_accuracy: 0.2781 - val_loss: 3.9369 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to model_init_2018-12-2708_18_49.548506/model-00001-5.03451-0.27753-3.93688-0.24000.h5
Epoch 2/10
67/67 [==============================] - 28s 421ms/step - loss: 2.2612 - categorical_accuracy: 0.2736 - val_loss: 2.0963 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2018-12-2708_18_49.548506/model-00002-2.26116-0.27363-2.09633-0.30000.h5
Epoch 3/10
67/67 [==============================] - 28s 421ms/step - loss: 1.7592 - categorical_accuracy: 0.3433 - val_loss: 1.1549 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_init_2018-12-2708_18_49.548506/model-00003-1.75917-0.34328-1.15488-0.49000.h5
Epoch 4/10
67/67 [==============================] - 29s 437ms/step - loss: 1.6257 - categorical_accuracy: 0.2786 - val_loss: 1.5620 - val_categorical_accuracy: 0.3200

Epo

### Changing Model with images per frame, width and height of the images Experiment [9]

In [16]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [17]:
model_e = Sequential()
model_e.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_e.add(Activation('relu'))
model_e.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Flatten())
model_e.add(Dense(512, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(classes, activation='softmax'))
model_e.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_15 (Conv3D)           (None, 30, 64, 64, 32)    896       
_________________________________________________________________
activation_15 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_16 (Conv3D)           (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_16 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 10, 22, 22, 64)    55360     
__________

In [18]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 5
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 5



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 40s 600ms/step - loss: 2.3581 - categorical_accuracy: 0.2149 - val_loss: 5.3811 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to model_init_2018-12-2708_18_49.548506/model-00001-2.35805-0.21493-5.38115-0.16000.h5
Epoch 2/20
65/67 [============================>.] - ETA: 1s - loss: 2.2635 - categorical_accuracy: 0.2092

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 41s 613ms/step - loss: 2.2555 - categorical_accuracy: 0.2129 - val_loss: 4.4151 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2018-12-2708_18_49.548506/model-00002-2.25866-0.21148-4.41509-0.16000.h5
Epoch 3/20
67/67 [==============================] - 24s 358ms/step - loss: 2.1614 - categorical_accuracy: 0.1990 - val_loss: 1.7849 - val_categorical_accuracy: 0.1800

Epoch 00003: saving model to model_init_2018-12-2708_18_49.548506/model-00003-2.16142-0.19900-1.78486-0.18000.h5
Epoch 4/20
67/67 [==============================] - 25s 374ms/step - loss: 2.0766 - categorical_accuracy: 0.2090 - val_loss: 1.9560 - val_categorical_accuracy: 0.2000

Epoch 00004: saving model to model_init_2018-12-2708_18_49.548506/model-00004-2.07660-0.20896-1.95596-0.20000.h5
Epoch 5/20
67/67 [==============================] - 27s 396ms/step - loss: 1.9846 - categorical_accuracy: 0.2438 - val_loss: 1.6872 - val_categorical_accuracy: 0.3000



### Model F Experiment [10]

In [19]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_21 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_22 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [20]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 30
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10

Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 6.8595 - categorical_accuracy: 0.2561

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 87s 1s/step - loss: 6.8264 - categorical_accuracy: 0.2572 - val_loss: 5.2584 - val_categorical_accuracy: 0.2900

Epoch 00001: saving model to model_init_2018-12-2708_18_49.548506/model-00001-6.84936-0.25641-5.25835-0.29000.h5
Epoch 2/30
67/67 [==============================] - 26s 395ms/step - loss: 3.4366 - categorical_accuracy: 0.2438 - val_loss: 1.3242 - val_categorical_accuracy: 0.4500

Epoch 00002: saving model to model_init_2018-12-2708_18_49.548506/model-00002-3.43656-0.24378-1.32420-0.45000.h5
Epoch 3/30
67/67 [==============================] - 29s 437ms/step - loss: 2.0743 - categorical_accuracy: 0.3035 - val_loss: 1.4278 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2018-12-2708_18_49.548506/model-00003-2.07433-0.30348-1.42776-0.30000.h5
Epoch 4/30
67/67 [==============================] - 27s 396ms/step - loss: 1.7408 - categorical_accuracy: 0.4030 - val_loss: 1.3609 - val_categorical_accuracy: 0.3000

Epo


Epoch 00029: saving model to model_init_2018-12-2708_18_49.548506/model-00029-0.64705-0.75124-0.65264-0.76000.h5
Epoch 30/30
67/67 [==============================] - 27s 396ms/step - loss: 0.7928 - categorical_accuracy: 0.6766 - val_loss: 0.6484 - val_categorical_accuracy: 0.7400

Epoch 00030: saving model to model_init_2018-12-2708_18_49.548506/model-00030-0.79279-0.67662-0.64836-0.74000.h5


### Model G Experiment [11]

In [21]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_25 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_25 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_26 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [22]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10
66/67 [============================>.] - ETA: 1s - loss: 4.8691 - categorical_accuracy: 0.2652

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 93s 1s/step - loss: 4.8757 - categorical_accuracy: 0.2612 - val_loss: 1.8970 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2018-12-2708_18_49.548506/model-00001-4.87111-0.26395-1.89698-0.38000.h5
Epoch 2/10
67/67 [==============================] - 28s 420ms/step - loss: 2.6849 - categorical_accuracy: 0.2836 - val_loss: 2.1399 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2018-12-2708_18_49.548506/model-00002-2.68486-0.28358-2.13988-0.40000.h5
Epoch 3/10
67/67 [==============================] - 28s 424ms/step - loss: 1.9323 - categorical_accuracy: 0.3234 - val_loss: 1.4312 - val_categorical_accuracy: 0.3900

Epoch 00003: saving model to model_init_2018-12-2708_18_49.548506/model-00003-1.93227-0.32338-1.43115-0.39000.h5
Epoch 4/10
67/67 [==============================] - 28s 421ms/step - loss: 1.9627 - categorical_accuracy: 0.2886 - val_loss: 1.3860 - val_categorical_accuracy: 0.4000

Epo

### Model H Experiment [12]

In [23]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_h = Sequential()

model_h.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_h.add(Flatten())

model_h.add(Dense(nb_dense[0], activation='relu'))
model_h.add(Dropout(0.5))

model_h.add(Dense(nb_dense[1], activation='relu'))
model_h.add(Dropout(0.5))

#softmax layer
model_h.add(Dense(nb_dense[2], activation='softmax'))
model_h.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_h.summary()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_29 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_23 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_30 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [24]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_h.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10
66/67 [============================>.] - ETA: 1s - loss: 5.2818 - categorical_accuracy: 0.2576

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 90s 1s/step - loss: 5.2455 - categorical_accuracy: 0.2587 - val_loss: 1.8305 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2018-12-2708_18_49.548506/model-00001-5.27065-0.25792-1.83051-0.37000.h5
Epoch 2/10
67/67 [==============================] - 28s 421ms/step - loss: 2.3795 - categorical_accuracy: 0.2687 - val_loss: 1.9366 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model_init_2018-12-2708_18_49.548506/model-00002-2.37949-0.26866-1.93659-0.28000.h5
Epoch 3/10
67/67 [==============================] - 29s 432ms/step - loss: 1.8573 - categorical_accuracy: 0.2289 - val_loss: 1.2823 - val_categorical_accuracy: 0.5000

Epoch 00003: saving model to model_init_2018-12-2708_18_49.548506/model-00003-1.85731-0.22886-1.28228-0.50000.h5
Epoch 4/10
67/67 [==============================] - 29s 429ms/step - loss: 1.5744 - categorical_accuracy: 0.3333 - val_loss: 1.2822 - val_categorical_accuracy: 0.4600

Epo

### Model I Experiment [13]

In [36]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_i = Sequential()

model_i.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(Activation('relu'))
model_i.add(Dropout(0.25))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_i.add(Flatten())

model_i.add(Dense(nb_dense[0], activation='relu'))
model_i.add(Dropout(0.5))

model_i.add(Dense(nb_dense[1], activation='relu'))
model_i.add(Dropout(0.5))

#softmax layer
model_i.add(Dense(nb_dense[2], activation='softmax'))
model_i.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_41 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_30 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_41 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_35 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_42 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_31 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_42 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [37]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_i.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 10
Epoch 1/10
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 2.2026 - categorical_accuracy: 0.3242

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 93s 1s/step - loss: 2.2024 - categorical_accuracy: 0.3195 - val_loss: 1.4507 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2018-12-2709_21_34.860565/model-00001-2.20253-0.32278-1.45066-0.28000.h5
Epoch 2/10
67/67 [==============================] - 27s 401ms/step - loss: 1.5683 - categorical_accuracy: 0.3085 - val_loss: 1.8319 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2018-12-2709_21_34.860565/model-00002-1.56827-0.30846-1.83194-0.20000.h5
Epoch 3/10
67/67 [==============================] - 30s 450ms/step - loss: 1.4762 - categorical_accuracy: 0.3383 - val_loss: 2.2867 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2018-12-2709_21_34.860565/model-00003-1.47623-0.33831-2.28666-0.25000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/10
67/67 [==============================] - 27s 406ms/step - loss: 1.3299 - categorical_accuracy: 0.

### Final Model

In [10]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [11]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_5 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_6 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_6 (Activation)    (None, 15, 60, 60, 16)    0         
__________

In [12]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [46]:
batch_size = 10
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  /storage/Demo_Gesture/Project_data/valSource path =  /storage/Demo_Gesture/Project_data/train ; batch size =  ; batch size = 10
10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 1.5198 - categorical_accuracy: 0.3045

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 95s 1s/step - loss: 1.5213 - categorical_accuracy: 0.3050 - val_loss: 1.5963 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2018-12-2709_21_34.860565/model-00001-1.52027-0.30468-1.59634-0.28000.h5
Epoch 2/20
67/67 [==============================] - 27s 402ms/step - loss: 1.5324 - categorical_accuracy: 0.2687 - val_loss: 1.4435 - val_categorical_accuracy: 0.4600

Epoch 00002: saving model to model_init_2018-12-2709_21_34.860565/model-00002-1.53241-0.26866-1.44352-0.46000.h5
Epoch 3/20
67/67 [==============================] - 29s 431ms/step - loss: 1.5477 - categorical_accuracy: 0.3483 - val_loss: 1.3526 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2018-12-2709_21_34.860565/model-00003-1.54766-0.34826-1.35262-0.44000.h5
Epoch 4/20
67/67 [==============================] - 30s 443ms/step - loss: 1.4832 - categorical_accuracy: 0.2935 - val_loss: 1.5432 - val_categorical_accuracy: 0.3300

Epo

In [13]:
batch_size = 32
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20Source path =  /storage/Demo_Gesture/Project_data/trainSource path =  /storage/Demo_Gesture/Project_data/val ; batch size = 
 ; batch size = 10
10


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 1s - loss: 2.0760 - categorical_accuracy: 0.2106

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 132s 2s/step - loss: 2.0679 - categorical_accuracy: 0.2075 - val_loss: 1.4500 - val_categorical_accuracy: 0.4000

Epoch 00001: saving model to model_init_2018-12-2809_14_05.984886/model-00001-2.07353-0.20965-1.45004-0.40000.h5
Epoch 2/20
67/67 [==============================] - 30s 454ms/step - loss: 1.6320 - categorical_accuracy: 0.2537 - val_loss: 1.5572 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2018-12-2809_14_05.984886/model-00002-1.63199-0.25373-1.55723-0.30000.h5
Epoch 3/20
67/67 [==============================] - 32s 471ms/step - loss: 1.5626 - categorical_accuracy: 0.2537 - val_loss: 1.4360 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2018-12-2809_14_05.984886/model-00003-1.56261-0.25373-1.43603-0.44000.h5
Epoch 4/20
67/67 [==============================] - 30s 449ms/step - loss: 1.5626 - categorical_accuracy: 0.3085 - val_loss: 1.4860 - val_categorical_accuracy: 0.3300

Ep

In [15]:
batch_size = 64
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 64
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 64


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Batch:  2 Index: 64


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 288s 4s/step - loss: 0.7238 - categorical_accuracy: 0.7199 - val_loss: 0.8679 - val_categorical_accuracy: 0.6424

Epoch 00001: saving model to model_init_2018-12-2809_14_05.984886/model-00001-0.73238-0.71452-0.86791-0.64241.h5
Epoch 2/20
67/67 [==============================] - 168s 3s/step - loss: 0.6806 - categorical_accuracy: 0.7329 - val_loss: 0.8305 - val_categorical_accuracy: 0.6800

Epoch 00002: saving model to model_init_2018-12-2809_14_05.984886/model-00002-0.67985-0.73287-0.83045-0.68000.h5
Epoch 3/20
67/67 [==============================] - 156s 2s/step - loss: 0.5746 - categorical_accuracy: 0.7805 - val_loss: 0.9897 - val_categorical_accuracy: 0.6750

Epoch 00003: saving model to model_init_2018-12-2809_14_05.984886/model-00003-0.57463-0.78051-0.98971-0.67500.h5
Epoch 4/20
67/67 [==============================] - 158s 2s/step - loss: 0.5510 - categorical_accuracy: 0.8016 - val_loss: 0.7652 - val_categorical_accuracy: 0.7250

Epoch 00

In [17]:
batch_size = 40
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /storage/Demo_Gesture/Project_data/val ; batch size = 40
Epoch 1/20
Source path =  /storage/Demo_Gesture/Project_data/train ; batch size = 40


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/67 [..............................] - ETA: 7:32 - loss: 0.1220 - categorical_accuracy: 0.9625Batch:  3 Index: 40


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 246s 4s/step - loss: 0.1449 - categorical_accuracy: 0.9592 - val_loss: 0.7256 - val_categorical_accuracy: 0.7208

Epoch 00001: saving model to model_init_2018-12-2809_14_05.984886/model-00001-0.14324-0.96058-0.72556-0.72083.h5
Epoch 2/20
67/67 [==============================] - 161s 2s/step - loss: 0.1451 - categorical_accuracy: 0.9611 - val_loss: 0.7605 - val_categorical_accuracy: 0.7150

Epoch 00002: saving model to model_init_2018-12-2809_14_05.984886/model-00002-0.14431-0.96137-0.76048-0.71500.h5
Epoch 3/20
67/67 [==============================] - 162s 2s/step - loss: 0.1430 - categorical_accuracy: 0.9596 - val_loss: 0.7225 - val_categorical_accuracy: 0.7450

Epoch 00003: saving model to model_init_2018-12-2809_14_05.984886/model-00003-0.14302-0.95961-0.72253-0.74500.h5
Epoch 4/20
67/67 [==============================] - 163s 2s/step - loss: 0.1428 - categorical_accuracy: 0.9587 - val_loss: 0.6893 - val_categorical_accuracy: 0.7350

Epoch 00